# Softmax回归
:label:`sec_softmax`

在 :numref:`sec_linear_regression` 中，我们介绍了线性回归，
并通过 :numref:`sec_linear_scratch` 从头开始实现，
并在 :numref:`sec_linear_concise` 中使用深度学习框架的高级API来完成繁重的工作。

当我们想要回答*多少？*或*多少个？*的问题时，回归是我们首选的工具。
如果你想预测一栋房子将以多少美元（价格）出售，
或者一支棒球队可能会赢多少场比赛，
或者一个病人在出院前将在医院住多少天，
那么你可能正在寻找一个回归模型。
然而，即使在回归模型中，
也有重要的区别。
例如，房子的价格永远不会是负数，并且变化可能是相对于其基价而言的。
因此，对价格取对数进行回归可能更有效。
同样，病人住院的天数是一个*离散非负*随机变量。
因此，最小二乘法也可能不是理想的方法。
这种类型的时间到事件建模伴随着许多其他复杂情况，
这些问题在一个称为*生存建模*的专门子领域中处理。

这里的目的不是让你感到不知所措，而是让你知道，
估计不仅仅是简单地最小化平方误差。
更广泛地说，监督学习不仅仅包括回归。
在本节中，我们关注*分类*问题，
我们将搁置*多少？*的问题，转而关注*哪个类别？*的问题。



* 这封电子邮件应该放在垃圾邮件文件夹还是收件箱？
* 这位客户更有可能注册还是不注册订阅服务？
* 这张图片描绘的是驴、狗、猫还是公鸡？
* Aston最有可能接下来观看哪部电影？
* 你接下来要阅读书中的哪一节？

通俗地说，机器学习从业者
用*分类*这个词来描述两个略有不同的问题：
(i) 我们只关心将示例硬分配到类别（类）的情况；
(ii) 我们希望进行软分配的情况，即评估每个类别适用的概率。
区分往往变得模糊，部分原因是，
即使我们只关心硬分配，
我们仍然使用进行软分配的模型。

更有甚者，有时不止一个标签可能是真的。
例如，一篇新闻文章可能同时涵盖
娱乐、商业和太空飞行的主题，
但不涉及医学或体育。
因此，将其单独归入上述任何一个类别都不是很有用。
这个问题通常被称为[多标签分类](https://en.wikipedia.org/wiki/Multi-label_classification)。
参见 :citet:`Tsoumakas.Katakis.2007` 的概述
以及 :citet:`Huang.Xu.Yu.2015`
对于图像标记的有效算法。

## 分类
:label:`subsec_classification-problem`

为了热身，让我们从
一个简单的图像分类问题开始。
在这里，每个输入由一个$2\times2$的灰度图像组成。
我们可以用一个标量表示每个像素值，
给我们四个特征$x_1, x_2, x_3, x_4$。
进一步假设每张图像属于
“猫”、“鸡”和“狗”这三个类别之一。

接下来，我们必须选择如何表示标签。
我们有两个明显的选择。
也许最自然的想法是
选择$y \in \{1, 2, 3\}$，
其中整数代表
$\{\textrm{狗}, \textrm{猫}, \textrm{鸡}\}$。
这是在计算机上存储此类信息的好方法。
如果这些类别之间有一些自然的顺序，
比如说如果我们试图预测
$\{\textrm{婴儿}, \textrm{幼儿}, \textrm{青少年}, \textrm{年轻成人}, \textrm{成人}, \textrm{老年人}\}$，
那么甚至可以将此视为
[序数回归](https://en.wikipedia.org/wiki/Ordinal_regression)问题
并保持标签的这种格式。
参见 :citet:`Moon.Smola.Chang.ea.2010` 对不同类型排名损失函数的概述
以及 :citet:`Beutel.Murray.Faloutsos.ea.2014` 提出的解决具有多个模式响应的贝叶斯方法。

一般来说，分类问题并不带有
类别之间的自然顺序。
幸运的是，统计学家很久以前就发明了一种
表示分类数据的简单方法：*独热编码*。
独热编码是一个向量，
其分量数量与我们的类别数量相同。
对应于特定实例类别的分量设为1，
所有其他分量设为0。
在我们的情况下，标签$y$将是一个三维向量，
$(1, 0, 0)$对应于“猫”，$(0, 1, 0)$对应于“鸡”，
$(0, 0, 1)$对应于“狗”：

$$y \in \{(1, 0, 0), (0, 1, 0), (0, 0, 1)\}.$$

### 线性模型

为了估计与所有可能类别相关的条件概率，
我们需要一个具有多个输出的模型，每个类别一个输出。
为了用线性模型解决分类问题，
我们需要与输出一样多的仿射函数。
严格来说，我们只需要少一个，
因为最后一个类别必须是$1$与其他类别之和的差，
但由于对称性的原因，
我们使用稍微冗余的参数化。
每个输出对应自己的仿射函数。
在我们的情况下，因为我们有4个特征和3个可能的输出类别，
我们需要12个标量来表示权重（带下标的$w$），
和3个标量来表示偏置（带下标的$b$）。这产生：

$$
\begin{aligned}
o_1 &= x_1 w_{11} + x_2 w_{12} + x_3 w_{13} + x_4 w_{14} + b_1,\\
o_2 &= x_1 w_{21} + x_2 w_{22} + x_3 w_{23} + x_4 w_{24} + b_2,\\
o_3 &= x_1 w_{31} + x_2 w_{32} + x_3 w_{33} + x_4 w_{34} + b_3.
\end{aligned}
$$

相应的神经网络图
如 :numref:`fig_softmaxreg` 所示。
就像在线性回归中一样，
我们使用单层神经网络。
由于每个输出 $o_1, o_2$, 和 $o_3$ 的计算
都依赖于每个输入 $x_1, x_2, x_3$, 和 $x_4$，
输出层也可以被描述为一个*全连接层*。

![Softmax回归是一个单层神经网络。](../img/softmaxreg.svg)
:label:`fig_softmaxreg`

为了更简洁的表示，我们使用向量和矩阵：
$\mathbf{o} = \mathbf{W} \mathbf{x} + \mathbf{b}$ 更适合数学和代码。
请注意，我们已将所有权重收集到一个$3 \times 4$的矩阵中，所有偏置
$\mathbf{b} \in \mathbb{R}^3$ 收集到一个向量中。

### Softmax
:label:`subsec_softmax_operation`

假设有一个合适的损失函数，
我们可以直接尝试最小化
$\mathbf{o}$ 与标签 $\mathbf{y}$ 之间的差异。
虽然事实证明，将分类
视为向量值回归问题的效果惊人得好，
但它仍存在以下不满意之处：

* 没有保证输出 $o_i$ 以我们期望概率行为的方式相加为1。
* 没有保证输出 $o_i$ 甚至是非负的，即使它们的输出相加为1，或者它们不超过1。

这两个方面使得估计问题难以解决，
并且解决方案非常容易受到异常值的影响。
例如，如果我们假设卧室数量与
某人购买房屋的可能性之间存在正线性相关关系，
那么当涉及到购买豪宅时，
概率可能会超过1！
因此，我们需要一种机制来“压缩”输出。

有许多方法可以实现这一目标。
例如，我们可以假设输出
$\mathbf{o}$ 是 $\mathbf{y}$ 的损坏版本，
其中损坏通过添加噪声 $\boldsymbol{\epsilon}$ 发生，
该噪声是从正态分布中抽取的。
换句话说，$\mathbf{y} = \mathbf{o} + \boldsymbol{\epsilon}$，
其中 $\epsilon_i \sim \mathcal{N}(0, \sigma^2)$。
这就是所谓的 [probit 模型](https://en.wikipedia.org/wiki/Probit_model)，
首先由 :citet:`Fechner.1860` 引入。
虽然吸引人，但它的效果不如 softmax 好，
也没有导致特别好的优化问题。

另一种实现这一目标的方法
（并确保非负性）是使用指数函数 $P(y = i) \propto \exp o_i$。
这确实满足了条件类概率
随着 $o_i$ 的增加而增加的要求，它是单调的，
并且所有概率都是非负的。
然后，我们可以通过将每个值除以其总和
来使这些值相加为1。
这个过程称为*归一化*。
将这两部分结合起来
给出了*softmax*函数：

$$\hat{\mathbf{y}} = \mathrm{softmax}(\mathbf{o}) \quad \textrm{其中}\quad \hat{y}_i = \frac{\exp(o_i)}{\sum_j \exp(o_j)}.$$
:eqlabel:`eq_softmax_y_and_o`

注意，$\mathbf{o}$ 的最大坐标
对应于根据 $\hat{\mathbf{y}}$ 最可能的类别。
此外，因为softmax操作
保留了其参数之间的顺序，
我们不需要计算softmax
就可以确定哪个类别被赋予了最高概率。因此，

$$
\operatorname*{argmax}_j \hat y_j = \operatorname*{argmax}_j o_j.
$$


softmax 的想法可以追溯到 :citet:`Gibbs.1902`，
他借鉴了物理学的思想。
追溯得更远一些，Boltzmann，
现代统计物理学之父，
利用这一技巧来模拟气体分子的能量状态分布。
特别是，他发现能量状态
在热力学系综（如气体中的分子）
中的普遍程度与 $\exp(-E/kT)$ 成比例。
这里，$E$ 是状态的能量，
$T$ 是温度，$k$ 是玻尔兹曼常数。
当统计学家谈论增加或减少
统计系统的“温度”时，
他们指的是改变 $T$
以有利于更低或更高的能量状态。
按照 Gibbs 的想法，能量等同于误差。
基于能量的模型 :cite:`Ranzato.Boureau.Chopra.ea.2007`
在描述深度学习问题时采用了这种观点。

### 向量化
:label:`subsec_softmax_vectorization`

为了提高计算效率，
我们在小批量数据中向量化计算。
假设我们得到了一个小批量 $\mathbf{X} \in \mathbb{R}^{n \times d}$
包含 $n$ 个维度为 $d$（输入数量）的例子。
此外，假设我们在输出中有 $q$ 个类别。
那么权重满足 $\mathbf{W} \in \mathbb{R}^{d \times q}$
偏置满足 $\mathbf{b} \in \mathbb{R}^{1\times q}$。

$$ \begin{aligned} \mathbf{O} &= \mathbf{X} \mathbf{W} + \mathbf{b}, \\ \hat{\mathbf{Y}} & = \mathrm{softmax}(\mathbf{O}). \end{aligned} $$
:eqlabel:`eq_minibatch_softmax_reg`

这将主要操作加速为
矩阵-矩阵乘积 $\mathbf{X} \mathbf{W}$。
此外，由于 $\mathbf{X}$ 的每一行
代表一个数据样本，
softmax 操作本身可以按行计算：
对于 $\mathbf{O}$ 的每一行，先对所有条目求指数，
然后通过总和归一化。
不过需要注意的是，必须小心避免
对大数求指数和对数，
因为这可能导致数值溢出或下溢。
深度学习框架会自动处理这种情况。

## 损失函数
:label:`subsec_softmax-regression-loss-func`

现在我们有了从特征 $\mathbf{x}$
到概率 $\mathbf{\hat{y}}$ 的映射，
我们需要一种方法来优化这种映射的准确性。
我们将依赖最大似然估计，
这正是我们在
:numref:`subsec_normal_distribution_and_squared_loss` 中
提供均方误差损失的概率论依据时遇到的方法。

### 对数似然

softmax 函数给我们一个向量 $\hat{\mathbf{y}}$，
我们可以将其解释为给定任何输入 $\mathbf{x}$ 的
每个类别的（估计）条件概率，
例如 $\hat{y}_1 = P(y=\textrm{cat} \mid \mathbf{x})$。
在以下内容中，我们假设对于具有特征 $\mathbf{X}$ 的数据集，
标签 $\mathbf{Y}$ 使用 one-hot 编码标签向量表示。
我们可以通过检查实际类别
根据我们的模型有多大概率来比较估计值与实际情况：

$$
P(\mathbf{Y} \mid \mathbf{X}) = \prod_{i=1}^n P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)}).
$$

我们允许因子分解
因为我们假设每个标签都是独立地
从其各自的分布 $P(\mathbf{y}\mid\mathbf{x}^{(i)})$ 中抽取的。
由于最大化项的乘积很麻烦，
我们取负对数以获得等效的最小化负对数似然问题：

$$
-\log P(\mathbf{Y} \mid \mathbf{X}) = \sum_{i=1}^n -\log P(\mathbf{y}^{(i)} \mid \mathbf{x}^{(i)})
= \sum_{i=1}^n l(\mathbf{y}^{(i)}, \hat{\mathbf{y}}^{(i)}),
$$

对于任何一对标签 $\mathbf{y}$
和模型预测 $\hat{\mathbf{y}}$
在 $q$ 类别中，损失函数 $l$ 为

$$ l(\mathbf{y}, \hat{\mathbf{y}}) = - \sum_{j=1}^q y_j \log \hat{y}_j. $$
:eqlabel:`eq_l_cross_entropy`

出于稍后解释的原因，
:eqref:`eq_l_cross_entropy` 中的损失函数
通常被称为*交叉熵损失*。
由于 $\mathbf{y}$ 是长度为 $q$ 的 one-hot 向量，
对所有坐标 $j$ 的求和在除了一个项外都消失。
注意，当 $\hat{\mathbf{y}}$ 是概率向量时，
损失 $l(\mathbf{y}, \hat{\mathbf{y}})$
被限制在 $0$ 以上：
没有一个条目大于 $1$，
因此它们的负对数不能低于 $0$；
只有在我们以*确定性*预测实际标签时，
$l(\mathbf{y}, \hat{\mathbf{y}}) = 0$。
对于任何有限的权重设置，这都不可能发生，
因为将 softmax 输出推向 $1$
需要将对应的输入 $o_i$ 推向无穷大
（或将所有其他输出 $o_j$ 对于 $j \neq i$ 推向负无穷大）。
即使我们的模型能够分配输出概率为 $0$，
在如此高信心下犯错误
也会导致无限损失（$-\log 0 = \infty$）。

### Softmax 和交叉熵损失
:label:`subsec_softmax_and_derivatives`

由于 softmax 函数
和相应的交叉熵损失非常常见，
值得更好地理解它们是如何计算的。
将 :eqref:`eq_softmax_y_and_o` 插入 :eqref:`eq_l_cross_entropy`
中损失的定义并使用 softmax 的定义，我们得到

$$
\begin{aligned}
l(\mathbf{y}, \hat{\mathbf{y}}) &=  - \sum_{j=1}^q y_j \log \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} \\
&= \sum_{j=1}^q y_j \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j \\
&= \log \sum_{k=1}^q \exp(o_k) - \sum_{j=1}^q y_j o_j.
\end{aligned}
$$

为了更好地理解发生了什么，
考虑对任何 logit $o_j$ 的导数。我们得到

$$
\partial_{o_j} l(\mathbf{y}, \hat{\mathbf{y}}) = \frac{\exp(o_j)}{\sum_{k=1}^q \exp(o_k)} - y_j = \mathrm{softmax}(\mathbf{o})_j - y_j.
$$

换句话说，导数是
我们的模型分配的概率
（通过 softmax 操作表达的）
与实际发生的情况（通过 one-hot 标签向量中的元素表达的）
之间的差异。
在这个意义上，它与我们在回归中看到的非常相似，
那里的梯度是观察 $y$ 与估计 $\hat{y}$ 之间的差异。
这不是巧合。
在任何指数族模型中，
对数似然的梯度恰好由这个项给出。
这个事实使得在实践中计算梯度变得很容易。

现在考虑我们不仅观察到单个结果
而是整个结果分布的情况。
我们可以像之前一样使用相同的表示形式来表示标签 $\mathbf{y}$。
唯一的区别是，而不是一个只包含二进制条目的向量，
比如 $(0, 0, 1)$，我们现在有一个通用的概率向量，
比如 $(0.1, 0.2, 0.7)$。
我们之前用于定义损失 $l$
在 :eqref:`eq_l_cross_entropy` 中使用的数学
仍然很好地工作，
只是解释稍微更一般。
它是标签分布的预期损失。
这种损失称为*交叉熵损失*，
并且它是分类问题中最常用的损失之一。
我们可以通过介绍信息理论的基础知识
来揭开名称的神秘面纱。
简而言之，它衡量了
相对于我们预测应发生的情况 $\hat{\mathbf{y}}$，
我们需要多少比特来编码我们看到的情况 $\mathbf{y}$。
我们在下面提供了非常基本的解释。
有关信息理论的更多详细信息，请参阅
:citet:`Cover.Thomas.1999` 或 :citet:`mackay2003information`。

## 信息理论基础
:label:`subsec_info_theory_basics`

许多深度学习论文使用信息理论的直觉和术语。
为了理解它们，我们需要一些共同的语言。
这是一个生存指南。
*信息理论*处理
量化数据中包含的信息量的问题。
这对我们压缩数据的能力施加了限制。
对于分布 $P$，其*熵* $H[P]$ 定义为：

$$H[P] = \sum_j - P(j) \log P(j).$$
:eqlabel:`eq_softmax_reg_entropy`

信息理论的一个基本定理指出
为了编码从分布 $P$ 随机抽取的数据，
我们至少需要 $H[P]$ “纳特”来编码它 :cite:`Shannon.1948`。
如果你想知道“纳特”是什么，
它是当使用基数 $e$ 而不是基数 2 的代码时的“比特”等效。
因此，一个纳特是 $\frac{1}{\log(2)} \approx 1.44$ 比特。

### 惊异

你可能想知道压缩与预测有什么关系。
想象一下，我们有一串数据想要压缩。
如果我们总是能轻松预测下一个标记，
那么这串数据就很容易压缩。
举一个极端的例子，流中的每个标记
始终取相同的值。
这是一个非常无聊的数据流！
而且不仅无聊，还很容易预测。
因为标记始终相同，
我们不需要传输任何信息
来传达流的内容。
容易预测，容易压缩。

然而，如果我们不能完美地预测每一个事件，
那么我们有时可能会感到惊讶。
当一个事件被赋予较低概率时，
我们的惊讶感更大。
Claude Shannon 认为 $\log \frac{1}{P(j)} = -\log P(j)$
来量化观察到一个事件 $j$ 时的*惊异*，
给它分配了一个（主观的）概率 $P(j)$。
在 :eqref:`eq_softmax_reg_entropy` 中定义的熵
则是当一个人分配了真正匹配数据生成过程的正确概率时
的*预期惊异*。

### 再谈交叉熵

因此，如果熵是在某人知道真实概率时经历的惊讶水平，
那么你可能想知道，交叉熵是什么呢？
从 $P$ 到 $Q$ 的交叉熵，记作 $H(P, Q)$，
是具有主观概率 $Q$ 的观察者
在看到实际上根据概率 $P$ 生成的数据时的预期惊讶。
这由 $H(P, Q) \stackrel{\textrm{def}}{=} \sum_j - P(j) \log Q(j)$ 给出。
最低的交叉熵发生在 $P=Q$ 时。
在这种情况下，从 $P$ 到 $Q$ 的交叉熵是 $H(P, P)= H(P)$。

简而言之，我们可以从两个角度来看待交叉熵分类目标：
(i) 最大化观察数据的似然性；
(ii) 最小化我们（从而也是比特数）
需要沟通标签所需的惊讶。

## 总结与讨论

在本节中，我们遇到了第一个非平凡的损失函数，
允许我们优化*离散*输出空间。
其设计的关键在于我们采取了概率方法，
将离散类别视为来自概率分布的实例。
作为附带效应，我们遇到了 softmax，
这是一种方便的激活函数，
可将普通神经网络层的输出
转换为有效的离散概率分布。
我们看到，当结合 softmax 时，
交叉熵损失的导数
与平方误差的行为非常相似；
即通过取预期行为与其预测之间的差异。
尽管我们只能浅尝辄止，
但我们遇到了令人兴奋的统计物理和信息理论的联系。

虽然这足以让你入门，
并希望能激起你的兴趣，
但我们并没有深入探讨。
特别是在计算考虑方面。
具体来说，对于任何具有 $d$ 个输入和 $q$ 个输出的全连接层，
参数化和计算成本是 $\mathcal{O}(dq)$，
这在实践中可能是禁止性的高。
幸运的是，通过近似和压缩，
可以降低将 $d$ 个输入转换为 $q$ 个输出的成本。
例如，Deep Fried Convnets :cite:`Yang.Moczulski.Denil.ea.2015`
使用置换、傅里叶变换和缩放的组合
将成本从二次降低到对数线性。
类似的技巧也适用于更高级的
结构矩阵近似 :cite:`sindhwani2015structured`。
最后，我们可以使用四元数般的分解
将成本降低到 $\mathcal{O}(\frac{dq}{n})$，
只要我们愿意在准确性和计算及存储成本之间进行权衡 :cite:`Zhang.Tay.Zhang.ea.2021`，
基于压缩因子 $n$。
这是一个活跃的研究领域。
使其具有挑战性的是，
我们不一定追求
最紧凑的表示
或最少的浮点运算次数，
而是寻求可以在现代 GPU 上最高效执行的解决方案。

## 练习

1. 我们可以更深入地探索指数族和 softmax 之间的联系。
    1. 计算 softmax 的交叉熵损失 $l(\mathbf{y},\hat{\mathbf{y}})$ 的二阶导数。
    1. 计算由 $\mathrm{softmax}(\mathbf{o})$ 给出的分布的方差，并显示它与上面计算的二阶导数相匹配。
1. 假设我们有三个等概率出现的类别，即概率向量为 $(\frac{1}{3}, \frac{1}{3}, \frac{1}{3})$。
    1. 如果我们尝试为其设计二进制编码，会出现什么问题？
    1. 你能设计一个更好的编码吗？提示：如果尝试对两个独立观察进行编码会发生什么？如果联合编码 $n$ 个观察呢？
1. 在对通过物理线路传输的信号进行编码时，工程师并不总是使用二进制编码。例如，[PAM-3](https://en.wikipedia.org/wiki/Ternary_signal) 使用三个信号电平 $\{-1, 0, 1\}$ 而不是两个电平 $\{0, 1\}$。你需要多少个三进制单位来传输范围在 $\{0, \ldots, 7\}$ 内的整数？为什么在电子学方面这可能是个更好的主意？
1. [Bradley--Terry 模型](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model) 使用逻辑模型来捕捉偏好。用户在苹果和橙子之间选择时
假设分数为 $o_{\textrm{apple}}$ 和 $o_{\textrm{orange}}$。我们的要求是较大的分数应导致选择相应项目的可能性更高，并且
得分最高的项目是最有可能被选择的 :cite:`Bradley.Terry.1952`。
    1. 证明 softmax 满足这一要求。
    1. 如果你想允许选择既不是苹果也不是橙子的默认选项怎么办？提示：现在用户有三个选择。
1. Softmax 得名于以下映射：$\textrm{RealSoftMax}(a, b) = \log (\exp(a) + \exp(b))$。
    1. 证明 $\textrm{RealSoftMax}(a, b) > \mathrm{max}(a, b)$。
    1. 你能把两者之间的差异做到多小？提示：不失一般性，你可以设 $b = 0$ 并且 $a \geq b$。
    1. 证明这对 $\lambda^{-1} \textrm{RealSoftMax}(\lambda a, \lambda b)$ 成立，前提是 $\lambda > 0$。
    1. 显示当 $\lambda \to \infty$ 时，我们有 $\lambda^{-1} \textrm{RealSoftMax}(\lambda a, \lambda b) \to \mathrm{max}(a, b)$。
    1. 构造一个类似的 softmin 函数。
    1. 将其扩展到两个以上的数字。
1. 函数 $g(\mathbf{x}) \stackrel{\textrm{def}}{=} \log \sum_i \exp x_i$ 有时也称为 [对数配分函数](https://en.wikipedia.org/wiki/Partition_function_(mathematics))。
    1. 证明该函数是凸的。提示：为此，使用第一导数相当于 softmax 函数的概率，并显示第二导数是方差。
    1. 显示 $g$ 是平移不变的，即 $g(\mathbf{x} + b) = g(\mathbf{x})$。
    1. 如果某些坐标 $x_i$ 非常大，会发生什么？如果它们都很小呢？
    1. 显示如果我们选择 $b = \mathrm{max}_i x_i$，我们会得到一个数值稳定的实现。
1. 假设我们有一个概率分布 $P$。假设我们选择另一个分布 $Q$ 使得 $Q(i) \propto P(i)^\alpha$ 对于 $\alpha > 0$。
    1. 哪种 $\alpha$ 的选择对应于加倍温度？哪种选择对应于减半温度？
    1. 如果我们让温度接近 $0$ 会发生什么？
    1. 如果我们让温度接近 $\infty$ 会发生什么？

[讨论](https://discuss.d2l.ai/t/46)